# 순서
1. 필요한 라이브러리 설치 (없다고 하는건 알아서 설치바람)
2. 순서대로 2번째까지는 한번만 실행하면 된다.
3. 3 ~ 5번 코드 블록은 연결 끊길때마다 인덱스를 바꿔가면서 반복 실행하면 된다. (확인용 코드는 귀찮으면 안해도 ㄱㅊ)
4. 끊길 때마다 지금까지 크롤링 된 데이터를 저장해야 함. -> 저장 양식 지켜서 저장

In [105]:
# 1번 블록
pip install BeautifulSoup4
pip install pandas

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: C:\Users\SSAFY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [52]:
# 2번 블록
import requests
from bs4 import BeautifulSoup
import urllib.parse

def get_rate(detail_info):
    rate_list = [0,0,0]
    try:
        rates = detail_info[0].find_all('div', attrs={'class' : 'jw-scoring-listing__rating'})
        for rate in rates:
            r = rate.a.get_text()
            r = r.rstrip()
            if r[-1] == '%':
                r = int(r[:-1])
                rate_list[0] = r
            else:
                if r[-1] == ')':  # popularity가 있다면
                    r, popularity = r.split('(')
                    popularity = popularity[:-1]
                    p = int(popularity[:-1])
                    if popularity[-1] == 'k':
                        p *= 10**3
                    elif popularity[-1] == 'm':
                        p *= 10**6
                    rate_list[2] = p           
                r = float(r.rstrip())
                # print(r)
                rate_list[1] = r
    except:
        rate_list = None
    # print('rate_list : ', rate_list)
    return rate_list

def get_genre(detail_info):
    try:
        genres = detail_info[1].find('div', attrs={'class' : 'detail-infos__value'}).get_text()
        genre_list = list(genres.split(','))
        for i in range(len(genre_list)):
            genre_list[i] = genre_list[i].strip()
        # print('genre : ',genre_list)
    except:
        genre_list = None
    return genre_list

def get_run_time(detail_info):
    run_time = 0
    try:
        run_times = detail_info[2].find('div', attrs={'class' : 'detail-infos__value'}).get_text()
        run_time_list = run_times.split(' ')
        for run in run_time_list:
            if 'min' in run:
                run_time += int(run[:-3])
            elif '분' in run:
                run_time += int(run[:-1])
            
            if 'hr' in run:
                run_time += int(run[:-2])*60
            elif '시간' in run:
                run_time += int(run[:-2])*60

        # print('run_time : ',run_time)
    except:
        run_time = None
    return run_time

def get_content_rating(detail_info):
    try:
        content_rating = detail_info[3].find('div', attrs={'class' : 'detail-infos__value'}).get_text()
        content_rating = int(content_rating)
    except:
        content_rating = None
    # print('content_rating : ',content_rating)
    return content_rating

def get_title(title_block):
    try:
        block = title_block.h1.get_text().rstrip()
        title, season_info = block.split('-')
        seasons = season_info.split(' ')
        title = title.rstrip()
        season = int(seasons[-1])
        # print('season :',season)
    except:
        title = title_block.h1.get_text().strip()
        season = None

    return title, season

def get_links(ott_box):
    result = []
    link = ott_box.a.get('href')
    # for link in links:
    # print(link)
    # link = str(link)
    a = link[32:]
    # print('a :',a)
    b = a.split('&')[0]
    print('b :', b)
    if 'netflix' in b or 'wavve' in b or 'watcha' in b or 'disney-plus' in b:
        ott_url = urllib.parse.unquote(b)
        result.append(ott_url)
    print('ott 링크 ?? ',result)
    return result

def get_otts(ott_box):
    print(f'================= ott 이름 찾기')
    otts = []
    ott = ott_box.find('picture', attrs={'class' : 'provider-icon'})
    # print(ott_lists)
    # for ott in ott_lists:
    name = ott.img['alt']
    print('ott 이름 : ',name)
    if name in ('Watcha', 'Netflix', 'wavve', 'Disney Plus'):
        otts.append(name)
    print('ott 확인 :', otts)
    return otts

def crawling(link):
    webpage = requests.get(link)
    soup = BeautifulSoup(webpage.content, 'html.parser')

    picture = soup.find('picture', attrs={'class': 'picture-comp title-poster__image'})
    try:
        image = picture.img['data-src']
    except:
        image = None
    # print('image : ',image)
    backdrops = soup.find('picture', attrs={'class': 'picture-comp'})
    try:
        backdrop = backdrops.img['src']
    except:
        backdrop = None
    # print('backdrop :', backdrop)

    # 컨텐츠 정보 - 평점, 장르, 러닝타임, 연령등급
    detail_info = soup.find_all('div', attrs={'class': 'detail-infos'})
    rate_list = get_rate(detail_info)
    genre_list = get_genre(detail_info)
    run_time = get_run_time(detail_info)
    content_rating = get_content_rating(detail_info)

    # 제목, 개봉연도
    title_block = soup.find('div', attrs={'class': 'title-block'})
    title, season = get_title(title_block)
    release_date = title_block.span.get_text().strip()
    release_date = int(release_date[1:-1])
    # print('release_date',release_date)

    # ott 링크
    ott_box = soup.find('div', attrs={'class': 'price-comparison__grid__row__holder'})
    ott_links = get_links(ott_box)
    otts = get_otts(ott_box)
    
    # 에피소드 개수
    content = soup.find('div', attrs={'class' : 'jw-info-box__container-content'})
    contents = content.find_all('h2', attrs={'class': 'detail-infos__subheading--label'})
    episode_cnt = None
    for h2 in contents[2:4]:
        subtitle = h2.get_text().rstrip()
        # print(subtitle)
        if subtitle[-1:] == '화':
            try:
                episode_cnt = int(subtitle[:-1])
                break
            except:
                episode_cnt = None
            # print('episode_cnt : ',episode_cnt)

    # 줄거리
    try:
        Synopsis = soup.find('p', attrs={'class' : 'text-wrap-pre-line mt-0'}).span.get_text()
    except:
        Synopsis = None
        
    # info = [title, release_date, rate_list, genre_list, run_time, content_rating]
    return title, season, release_date, rate_list, genre_list, run_time, content_rating, episode_cnt, Synopsis, image, backdrop, ott_links, otts


# 크롤링 할당량
### 끊긴 시점부터 시작 인덱스만 바꿔주세용
**!!!!!!마지막으로 print된 인덱스부터 시작!!!!!**
- 병진 : netflix [0:4000]
- 준형 : watcha [0:4000]
- 은성 : wavve [0:4000]
- 용찬 : watcha [4000:]
- 민혁 : wavve [4000:8000]
- 성은 : netflix [4000:] + wavve [8000:]

In [53]:
# 3번 블록
# justwatch에서 자주 연결을 끊기 때문에 print 된 마지막 index부터 다시 시작하면 됩니다!
import csv

file = open('./dataset/links.csv', 'r', encoding='utf-8')
URLs = csv.reader(file)
URLs = list(URLs)
title = []
season = []
release_date = []
rate_jw = []
rate_imdb = []
popularity = []
genres = []
run_time = []
content_rating = []
episode_cnt = []
Synopsis = []
images = []
backdrops = []
otts = []
links = []
for index, url in URLs[2000:2003]:  # 이전 마지막 index부터 다시 크롤링 시작 (!!!!!!마지막으로 print된 인덱스부터!!!!!)
    print(f'============================================= {index} ==============================================') # 이걸로 마지막 인덱스 파악 ㄱㄱ
    webpage = requests.get(url)
    soup = BeautifulSoup(webpage.content, 'html.parser')
    content = soup.find('div', attrs={'class' : 'jw-info-box__container-content'})
    contents = content.find_all('h2', attrs={'class': 'detail-infos__subheading--label'})
    subtitle_list = []
    season_cnt = 0
    for h2 in contents:
        subtitle = h2.get_text().rstrip()
        # print(subtitle)
        if '시즌' in subtitle:
            # print(subtitle[:-4])
            season_cnt = int(subtitle[:-4]) 
            # print('season_cnt : ',season_cnt)
            break
        subtitle_list.append(h2.get_text())

    if season_cnt > 0:
        print('======================= 시즌제 컨텐츠')
        num = 1
        while num < season_cnt + 1:
            try:
                link = url + f'/시즌-{num}'
                # print(link)
                t, ss, release, rate_list, genre_list, run, rating, episode, Syno, img, backdrop, ott, link = crawling(link)
                print('결과 : ',t, ss, release, rate_list, genre_list, run, rating, episode, Syno, img, backdrop, ott, link)
                title.append(t)
                season.append(ss)
                release_date.append(release)        
                rate_jw.append(rate_list[0])
                rate_imdb.append(rate_list[1])
                popularity.append(rate_list[2])
                genres.append(genre_list)
                run_time.append(run)
                content_rating.append(rating)
                episode_cnt.append(episode)
                Synopsis.append(Syno)
                images.append(img)
                backdrops.append(backdrop)
                otts.append(ott)
                links.append(link)
                num += 1
            except:
                break
    else:
        print('======================== 단일 컨텐츠')
        t, ss, release, rate_list, genre_list, run, rating, episode, Syno, img, backdrop, ott, link = crawling(url)
        print('결과 : ',t, ss, release, rate_list, genre_list, run, rating, episode, Syno, img, backdrop, ott, link)
        title.append(t)
        season.append(ss)
        release_date.append(release)
        rate_jw.append(rate_list[0])
        rate_imdb.append(rate_list[1])
        popularity.append(rate_list[2])
        genres.append(genre_list)
        run_time.append(run)
        content_rating.append(rating)
        episode_cnt.append(episode)
        Synopsis.append(Syno)
        images.append(img)
        backdrops.append(backdrop)
        otts.append(ott)
        links.append(link)

============================================= 1999 ==============================================
======================== 단일 컨텐츠
b : http%3A%2F%2Fwww.netflix.com%2Ftitle%2F81425050
ott 링크 ??  ['http://www.netflix.com/title/81425050']
================= ott 이름 찾기
ott 이름 :  Netflix
ott 확인 : ['Netflix']
결과 :  디보션 None 2022 [79, 6.6, 19000] ['전쟁', '액션', '드라마'] 139 None None 미 해군 최초의 흑인 파일럿 제시 브라운. 한국 전쟁에서 위험을 무릅쓴 비행에 나선다. 그리고 그의 곁에는 생사를 함께하는 윙맨 톰 허드너가 있다. https://images.justwatch.com/poster/304841860/s592/devotion https://www.justwatch.com/images/backdrop/302276499/s1440/devotion ['http://www.netflix.com/title/81425050'] ['Netflix']
============================================= 2000 ==============================================
======================== 단일 컨텐츠
b : http%3A%2F%2Fwww.netflix.com%2Ftitle%2F70020509
ott 링크 ??  ['http://www.netflix.com/title/70020509']
================= ott 이름 찾기
ott 이름 :  Netflix
ott 확인 : ['Netflix']
결과 :  히트 None 1995 [89, 8.3, 670000] ['드라마', '스릴러', '범죄', '액션

In [26]:
# 4번 블록 (확인용)
for info in zip(title,season, release_date,rate_jw,rate_imdb,popularity,genres,run_time,content_rating ,episode_cnt,Synopsis,images,backdrops):
    print(info)


# 저장하기 전
현재 위치 (이 파일을 저장하고 실행한 위치)에 dataset이라는 폴더를 만드새용 (1번만 하면 됨!)
만들기만 하고 아래 5번 블록 실행하면 됨 good

In [25]:
# 5번 블록
# csv 파일로 저장하는 코드 
# ***** 필수 *****
# 현재 실행 위치(이 크롤링 파일 저장한 위치)에 dataset 폴더 만들고 시작하기

import pandas as pd
import os
savePath = os.getcwd() +'\\dataset'

df = pd.DataFrame({"title": title, "season": season, "release_date": release_date, "rate_jw": rate_jw, "rate_imdb": rate_imdb, "popularity": popularity, "genres": genres, "run_time": run_time, "content_rating": content_rating , "episode_cnt": episode_cnt, "Synopsis": Synopsis, "images": images, "backdrops": backdrops})

# 본인이 맡은 파트의 ott 이름 + 시작 index + 끝 index 형식으로 파일 저장
df.to_csv(savePath+"\\wavve_10273_end.csv", index=True, encoding='utf-8-sig')